# VI: Second Practical Work

**Authors:** Gerard Comas & Marc Franquesa.

In [ ]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd
import warnings

alt.data_transformers.disable_max_rows()
warnings.simplefilter(action="ignore", category=FutureWarning)

collisions = pd.read_csv("./processed-data/collisions_weather.csv")
map_data = gpd.read_file("./processed-data/map.geojson")


primary = "green"
secondary = ""

colors = {
    primary: "#66c2a4",
    secondary: "",
}

collisions["VALID"] = collisions["VALID"].astype("int64")

collisions.head()

La nostre idea per a respondre a totes les preguntes d'aquest segon projecte es crear un Dashboard interactiu que tindrà la distribució següent:
- En la primera fila tres barplots en el que es representen el total de colisions amb un filtre concret. Aquests tres barplots seràn independents. Els tres barplots compartiran l'escala i l'escala de colors per tal de poder apreciar millor les magnituds. 
    - El primer de tots serà un barplot amb el total de colisions per cada mes, amb la funcionalitat de poder seleccionat una de les barres i els gràfics de les altre files dependeràn d'aquest filtrat. Si no es selecciona cap es mostraran tots els mesos. 
    - El segon serà un barplot amb el total de colisions per cada tipus de vehicle, amb la mateixa funcionalitat que abans.
    - El tercer i últim serà un barplot amb el total de colisions per a cada condició climàtica, amb la mateia funcionalitat que abans. 
- En la segona fila trobarem un element:
    - Un mapa de New York, on primerament es mostrin les fronteres dels barris. Aquest mapa dera un chloropleth i l'intensitat del color indicarà el total de colisions pel filtrat seleccionat. 
        - Hem de decidir si l'escala serà independent o no del total
        - També, podriem afegir més granularitat temporal amb un desplegable per a canviar.
        - El mapa incorporarà un tooltip que ens identifiqui la localitació i també el nombre de colisions
        - A més, serà interactiu i pemetrà filtrar per als gràfics de les lines inferiors.
- En la tercera fila trobarem dos elements:
    - Un heatmap on les files indiquen les setmanes i a les columnes els dies de la setmana. Està dissenyat per a que recordi a un calendari. Quan no s'hagi seleccionat el mes es mostrarà unes opcions per a escollir el mes a representar. El color indicarà la quantitat de colisions aquell dia.
    - En el segon element trobarem un line chart, aquest mostrarà totes les línes per a cada area dels mesos seleccionats, si s'ha seleccionat un mes, només es mostrarnan la d'aquell mes i si s'ha seleccionat una area es resaltarà aquella linea per sobre les altres. A més, hi haurà un slider per a marcar quina hora volem analitar, indicant-nos amb punts sobre les lines els valors per a cada area i per tant poder obtenir els valors més facilment. 

Pot ser confús per l'usuari doncs no queda del tot clar quines gràfiques filtren i quines no igual que la dependencia entre gràfics. 

In [ ]:
# TODO: comentar ordres

month_order = ["June", "July", "August", "September"]
month_selection = alt.selection_point(fields=["MONTH"], empty=True)

vehicle_order = ["Taxi", "Ambulance", "Fire truck"]
vehicle_selection = alt.selection_point(fields=["VEHICLE"], empty=True)

weather_order = ["Rainy", "Clear", "Partly cloudy", "Cloudy"]
weather_selection = alt.selection_point(fields=["WEATHER"], empty=True)

bars_df = collisions.groupby(["MONTH", "VEHICLE", "VEHICLE EMOJI", "WEATHER", "WEATHER EMOJI"]).agg({"VALID": "sum"}).reset_index()
months = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("MONTH:O", sort=month_order, axis=alt.Axis(title="Month", labelAngle=0)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(month_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        month_selection
    ).transform_filter(
        vehicle_selection & weather_selection
    )
    .properties(
        title=alt.Title(["Collisions per Month", "(filtered by vehicle and weather)"], dy=-0),
        width=250,
        height=175
    )
)

vehicles = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("VEHICLE:N", sort=vehicle_order, axis=alt.Axis(title="Vehicle", labels=False, domain=False, ticks=False, grid=False)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    ).transform_filter(
        month_selection & weather_selection
    )
    .properties(
        title=alt.Title(["Collisions per Vehicle", "(filtered by month and weather)"], dy=-0),
        width=200,
        height=175
    )
)

vehicles += (
    alt.Chart(bars_df)
    .mark_text(size=18, align="center", dy=-8)
    .encode(
        x=alt.X("VEHICLE:N", sort=vehicle_order),
        y=alt.Y("sum(VALID):Q"),
        text=alt.Text("VEHICLE EMOJI:N"),
        opacity=alt.condition(vehicle_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("VEHICLE:N", title="Vehicle"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        vehicle_selection
    ).transform_filter(
        month_selection & weather_selection
    )
)

weather = (
    alt.Chart(bars_df)
    .mark_bar(color=colors[primary])
    .encode(
        x=alt.X("WEATHER:N", sort=weather_order, axis=alt.Axis(title="Weather", labels=False, domain=False, ticks=False, grid=False)),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(weather_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("WEATHER:N", title="Weather"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        weather_selection
    ).transform_filter(
        month_selection & vehicle_selection
    ).properties(
        title=alt.Title(["Collisions per Weather", "(filtered by month and vehicle)"], dy=-0),
        width=200,
        height=175
    )
)

weather += (
    alt.Chart(bars_df)
    .mark_text(size=18, align="center", dy=-8)
    .encode(
        x=alt.X("WEATHER:N", sort=weather_order),
        y=alt.Y("sum(VALID):Q"),
        text=alt.Text("WEATHER EMOJI:N"),
        opacity=alt.condition(weather_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("WEATHER:N", title="Weather"), alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).add_params(
        weather_selection
    ).transform_filter(
        month_selection & vehicle_selection
    )
)

In [ ]:
(months & (vehicles | weather))

Now we will add an interactive map of NY

In [ ]:
map_data.head()

In [ ]:
ny_map_selection = alt.selection_point(fields=["BOROUGH"], empty=True)

collisions_borough=collisions.groupby(["MONTH","VEHICLE","WEATHER","BOROUGH"]).agg({"VALID": "sum"}).reset_index()
map_data = map_data[["BOROUGH", "AREA_KM2", "geometry"]]

ny_map = (
    alt.Chart(collisions_borough)
    .mark_geoshape(stroke="gray")
    .project(type="albersUsa")
    .transform_lookup(
        lookup = "BOROUGH",                 
        from_ = alt.LookupData(data=map_data, key="BOROUGH", fields=["geometry", "type", "AREA_KM2"]), 
    )
    .transform_filter(month_selection & weather_selection & vehicle_selection)
    .transform_aggregate(sumCollisions = "sum(VALID)", groupby = ["BOROUGH", "AREA_KM2", "geometry", "type"])
    .transform_calculate(COLLISIONS_KM2 = "datum.sumCollisions / datum.AREA_KM2")
    .encode(
        color=alt.condition(ny_map_selection, alt.Color("COLLISIONS_KM2:Q", scale=alt.Scale(scheme='greens'), legend=alt.Legend(title="Collisions per km2")), alt.value("lightgray")),
        tooltip=[alt.Tooltip("BOROUGH:N", title="Borough"), alt.Tooltip("COLLISIONS_KM2:Q", title="Collisions per km2")],
    ).properties(
        width=300,
        height=300,
        title=["NYC Boroughs", "(filtered by barplots)"]
    ).add_params(
        ny_map_selection
    )
)    

In [ ]:
((months | vehicles | weather) & (ny_map)).resolve_legend(color="independent")

Weekdays

In [ ]:
weekdayorder = ["Mon", "Tue", "Wed", "Thu", "Fri", "Sat", "Sun"]

# Default Mon to make it "quicker" to answer Q3
day_selection = alt.selection_point(fields=["CRASH WEEKDAY"], value="Mon")

weekdays_df = collisions.groupby(["CRASH DAY", "CRASH WEEKDAY", "CRASH WEEK NUMBER", "MONTH", "VEHICLE", "WEATHER", "DAY", "BOROUGH"]).agg({"VALID": "sum"}).reset_index()
weekdays = (
    alt.Chart(weekdays_df)
    .mark_rect()
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & ny_map_selection
    )
    .encode(
        x=alt.X("CRASH WEEKDAY:O", title="Day of week", sort=weekdayorder, axis=alt.Axis(labelAngle=0)),
        y=alt.Y("CRASH WEEK NUMBER:O", title="Week of year"),
        color=alt.Color("sumValid:Q", scale=alt.Scale(scheme="greens"), title="Collisions"),
        opacity=alt.condition(day_selection, alt.value(1), alt.value(0.2)),
        tooltip=[alt.Tooltip("CRASH DAY:O", title="Crash day"), alt.Tooltip("sumValid:Q", title="Collisions")],
    )
    .transform_aggregate(
        groupby=["CRASH DAY", "CRASH WEEKDAY", "CRASH WEEK NUMBER"],
        sumValid="sum(VALID):Q",
    )
    .add_params(
        day_selection
    ).properties(
        title=["Collisions per Week and Weekday", "(filtered by barplots and map)"],
        width=300,
        height=300,
    )
)

weekdays += (
    alt.Chart(weekdays_df)
    .mark_text(align="center", text="*", color="white", dy=3, size=15)
    .encode(
        x=alt.X("CRASH WEEKDAY:O", sort=weekdayorder),
        y=alt.Y("CRASH WEEK NUMBER:O"),
    )
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & ny_map_selection
    )
    .transform_aggregate(
        groupby=["CRASH DAY", "CRASH WEEKDAY", "CRASH WEEK NUMBER"],
        sumValid="sum(VALID):Q",
    )
    .transform_window(
        sort=[alt.SortField(field="sumValid", order="descending")],
        rank="rank()",
    )
    .transform_filter(alt.datum.rank == 1)
)

# Text, not convincing
# weekdays += (
#     alt.Chart(collisions)
#     .mark_text(align="left", size=10)
#     .transform_filter(month_selection)
#     .transform_aggregate(groupby = ["DAY", "MONTH", "CRASH WEEKDAY", "CRASH WEEK NUMBER"])
#     .encode(
#         x=alt.X("CRASH WEEKDAY:O", title=None, sort=weekdayorder),
#         y=alt.Y("CRASH WEEK NUMBER:O", title=None),
#         text=alt.Text("DAY:N") 
#     )
# )

In [ ]:
((months | vehicles | weather) & (ny_map | weekdays).resolve_legend(color="independent").resolve_scale(color="independent"))

Line chart

In [ ]:
hour_selection = alt.selection_point(encodings=["x"], nearest=True, value=12)

hours_df = collisions.groupby(["CRASH DAY", "CRASH WEEKDAY", "MONTH", "VEHICLE", "WEATHER", "BOROUGH", "DAY", "HOUR", "LOCATION AT HOUR"]).agg({"VALID": "sum"}).reset_index()

hours = (
    alt.Chart(hours_df)
    .mark_line()
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(title="Hour", labelAngle=0), scale=alt.Scale(domain=[0, 23])),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Collisions")),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
        color=alt.Color("BOROUGH:N", legend=alt.Legend(title="Borough")),
        tooltip=[alt.Tooltip("sum(VALID):Q", title="Collisions")],
    ).properties(
        title=["Collisions per Hour and Location (filtered by barplots and heatmap)", ""],
        width=700,
        height=300
    )
)

hours += (
    alt.Chart(hours_df)
    .mark_text(color="black", size=10, clip=False, align="left", dy=-10, dx=5)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection & ~hour_selection
    )
    .encode(
        x=alt.X("HOUR:Q", sort=weekdayorder, scale=alt.Scale(domain=[0, 23])),
        y=alt.Y("sumValid:Q"),
        text="LOCATION AT HOUR:N",
        color=alt.Color("BOROUGH:N", legend=None),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
    )
    .transform_aggregate(
        groupby=["BOROUGH", "HOUR", "LOCATION AT HOUR"],
        sumValid="sum(VALID):Q",
    )
    .transform_window(
        sort=[alt.SortField(field="sumValid", order="descending")],
        rank="rank()",
    )
    .transform_filter(alt.datum.rank == 1)
)

hours += (
    alt.Chart(hours_df)
        .mark_circle(opacity=0, size=50)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection & ~hour_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(labelAngle=0), scale=alt.Scale(domain=[0, 23])),
        y=alt.Y("sumValid:Q"),
        color=alt.Color("BOROUGH:N"),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
    )
    .transform_aggregate(
        groupby=["BOROUGH", "HOUR", "LOCATION AT HOUR"],
        sumValid="sum(VALID):Q",
    )
    .transform_window(
        sort=[alt.SortField(field="sumValid", order="descending")],
        rank="rank()",
    )
    .transform_filter(alt.datum.rank == 1)
)


hours_tooltip = (
    alt.Chart(hours_df)
    .mark_rule(color="gray", strokeDash=[10, 10])
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection & hour_selection
    )
    .encode(
        x=alt.X("HOUR:Q", scale=alt.Scale(domain=[0, 23])),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
    )
)

hours_tooltip += (
    alt.Chart(hours_df)
    .mark_circle(opacity=0, size=50)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection & hour_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(labelAngle=0), scale=alt.Scale(domain=[0, 23])),
        y=alt.Y("sum(VALID):Q", axis=alt.Axis(title="Total collisions")),
        color=alt.Color("BOROUGH:N"),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
    )
)

hours_tooltip += (
    alt.Chart(hours_df)
    .mark_text(align="left", dx=5, dy=-5, size=10)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & day_selection
    )
    .encode(
        x=alt.X("HOUR:Q", axis=alt.Axis(labelAngle=0), scale=alt.Scale(domain=[0, 23])),
        y=alt.Y("sum(VALID):Q"),
        text=alt.condition(hour_selection, "sum(VALID):Q", alt.value('')),
        opacity=alt.condition(ny_map_selection, alt.value(1), alt.value(0.2)),
        color=alt.Color("BOROUGH:N"),
    )
    .add_params(
        hour_selection
    )
)

In [ ]:
(
    (months | vehicles | weather) &
    (ny_map | weekdays).resolve_legend(color="independent").resolve_scale(color="independent") &
    (hours + hours_tooltip)
).resolve_legend(color="independent")

### Extra

Ens interesa saber com afecten les diferents caracteristiques al nombre de ferits i morts, també alguna infomarció sobre el tipus d'accident, doncs per l'estudi fet anteriorment sabem que hi ha accidents més comuns depenent del tipus però no vam arribar a estudiar la mortalitat de cadascun

In [ ]:
factor_df = collisions.groupby(["CRASH DAY", "CRASH WEEKDAY", "MONTH", "VEHICLE", "WEATHER", "BOROUGH", "ORIGINAL FACTOR", "FACTOR" ]).agg({"VALID": "sum", "NUMBER OF PERSONS INJURED": "sum",
    "NUMBER OF PERSONS KILLED": "sum"}).reset_index()

factor_df["NUMBER OF PERSONS KILLED"].sum()


In [ ]:
factor_df = factor_df[factor_df["FACTOR"] == "Driving Infraction"]

factor_df["NUMBER OF PERSONS KILLED"].sum()

In [ ]:
factors = (
    alt.Chart(factor_df)
    .mark_circle(color=colors[primary], size=100)
    .transform_filter(
        month_selection & weather_selection & vehicle_selection & ny_map_selection
    )
    .transform_aggregate(
        sumValid="sum(VALID):Q",
        sumInjured="sum(NUMBER OF PERSONS INJURED):Q",
        groupby=["ORIGINAL FACTOR"],
    )
    .transform_calculate(
        INJURED_PER_COLLISION = "datum['sumInjured'] / datum['sumValid']"
    )
    .encode(
        x=alt.X("INJURED_PER_COLLISION:Q", axis=alt.Axis(title="Injuries per collision", tickCount=10)),
        y=alt.Y("sumValid:Q", axis=alt.Axis(title="Collisions")),
        tooltip=[alt.Tooltip("ORIGINAL FACTOR:N", title="Factor"), alt.Tooltip("sumValid:Q", title="Collisions"), alt.Tooltip("INJURED_PER_COLLISION:Q", title="Injuries per collision")],
    ).properties(width=700, height=300)

)

(
    (months | vehicles | weather) &
    (ny_map | factors).resolve_legend(color="independent").resolve_scale(color="independent") &
    (weekdays | (hours + hours_tooltip)) &
    (factors)
).resolve_legend(color="independent")